In [1]:
PICKLE_CASE = "case8"
CASE = "case26"

# Import

In [2]:
import pandas as pd
import numpy as np
from local_para import *

In [3]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

# Set SEED

In [4]:
# seed固定
import os, sys, gc, warnings, random, datetime

def seed_everything(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
SEED = 2020
seed_everything(SEED)

# Read Data

In [5]:
TRAIN_ORG_PATH

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/sinchir02/code/Users/td017/kaggle-pipeline/input/train.csv'

In [6]:
TEST_ORG_PATH

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/sinchir02/code/Users/td017/kaggle-pipeline/input/test.csv'

In [7]:
train = pd.read_csv(TRAIN_ORG_PATH)
test = pd.read_csv(TEST_ORG_PATH)

SAME_PATH = "/mnt/batch/tasks/shared/LS_root/mounts/clusters/sinchir02/code/"
category = pd.read_csv(f"{SAME_PATH}" + "Users/td017/kaggle-pipeline/input/category.csv")
jan = pd.read_csv(f"{SAME_PATH}" + "Users/td017/kaggle-pipeline/input/jan.csv")
meta = pd.read_csv(f"{SAME_PATH}" + "Users/td017/kaggle-pipeline/input/meta.csv")
purchase_log = pd.read_csv(f"{SAME_PATH}" + "Users/td017/kaggle-pipeline/input/purchase_log.csv")
sample_submission = pd.read_csv(f"{SAME_PATH}" + "Users/td017/kaggle-pipeline/input/sample_submission.csv")

In [8]:
train.head(10)

,purchase_id,130123,130125,130129,130131,140307,140313,140316,140317,140321,140501,140505,140641,140691
0,njibeyLPrsnu4HCopjBihW,0,0,0,0,0,0,0,0,0,0,0,0,0
1,FqBfZgvrWVNMsCqGmZMdv3,0,0,1,0,0,0,0,0,0,0,0,0,0
2,KYE5JJ4y6zJBipkCKobwVg,0,0,0,0,0,0,0,0,0,0,0,0,0
3,tC5JqjsVxsKxQ8Ykk9S7fg,0,0,0,1,0,0,0,0,0,0,0,0,0
4,Pigjc37smwP2E3Z4VtKinB,0,0,0,0,0,0,0,0,0,0,1,0,0
5,YCHTGgFS6shv3GCMB7sB5j,0,0,0,1,0,0,0,0,0,0,0,0,0
6,nQsjoHBDtiJvKNUxzUoR4d,0,0,0,0,0,0,0,0,0,0,0,0,0
7,LXdStjW5USNHWpcXHd4E5j,0,0,0,0,0,0,1,0,0,0,0,0,0
8,9qw6QFmtqjjbPS9pMrwi7S,0,0,1,0,0,0,0,0,0,0,0,0,0
9,e6qaQuMJ3xsZJ96QmgCnxN,0,0,0,0,0,0,0,0,0,0,0,0,0


In [9]:
test.head()

,purchase_id
0,C3rcdjjRyw9qSh6NcZMKSX
1,Y56uzwqQzynHYZ4bDfLPp5
2,xDMdFERmC7CD9yFvyvKJnh
3,zzZENdjz7SvUQkGZV45afF
4,zFWkhHbLYJ9Fh5kUvCrx4g


In [ ]:
category.head()

In [ ]:
jan.head()

In [ ]:
meta.head()

In [ ]:
purchase_log.head()

In [ ]:
sample_submission.head()

# EDA

### ccl_jan

In [ ]:
purchase_log.head()

In [ ]:
purchase_log["ccl_jan"]

In [ ]:
ccl_jan_list = []

for i in purchase_log["ccl_jan"].tolist():
    if len(str(i)) == 13:
        ccl_jan_list.append(str(i)[:7])
    else:
        ccl_jan_list.append(np.nan)

In [ ]:
purchase_log["ccl_jan_company"] = ccl_jan_list

In [ ]:
purchase_log["ccl_jan_company"] .value_counts().head(10)

# Data Merge

In [ ]:
purchase_merge = purchase_log.merge(jan,on=['ccl_jan','ccl_category_cd4'])
purchase_merge = purchase_merge.merge(category,on='ccl_category_cd4')
purchase_merge = purchase_merge.merge(meta,on='purchase_id')
#train_df = train.merge(purchase_merge,on='purchase_id')
#test_df = test.merge(purchase_merge,on='purchase_id')

In [ ]:
purchase_merge = purchase_merge[~purchase_merge['ccl_category_cd4'].isin(sample_submission.columns)]

In [ ]:
purchase_merge.head()

In [ ]:
purchase_merge.to_pickle("/mnt/batch/tasks/shared/LS_root/mounts/clusters/sinchir02/code/Users/td017/kaggle-pipeline/tmp/purchase_merge.csv")

In [ ]:
train_meta = train.merge(meta,on='purchase_id')
test_meta = test.merge(meta,on='purchase_id')

# Feature Engineering

In [ ]:
train_tmp = train_meta.copy()
test_tmp = test_meta.copy()

# Time

In [ ]:
#!pip install jpholiday

In [ ]:
# ref
# https://www.guruguru.ml/competitions/9/discussions/5e86691f-1f18-4860-a52a-df79e7d6697b

import datetime
import jpholiday

def isHoliday(date):
    if date.weekday() >= 5 or jpholiday.is_holiday(date):
        return 1
    else:
        return 0

def isSatSun(date):
    if date.weekday() >= 5:
        return 1
    else:
        return 0

date = pd.Series(pd.to_datetime(meta["p_date"].value_counts().sort_index().index))

#特徴量生成
years = date.apply(lambda x: x.year)
months = date.apply(lambda x: x.month)
days = date.apply(lambda x: x.day)
weekdays = date.apply(lambda x: x.weekday())
holidays = date.apply(lambda x: isHoliday(x))
satsuns = date.apply(lambda x: isSatSun(x))
specials = date.apply(lambda x: jpholiday.is_holiday(x)).astype(int)

#大型連休フラグ

#バレンタインフラグ

time_df = pd.concat([date, years, months, days, weekdays, holidays, satsuns, specials], axis=1)
time_df.columns = ["p_date", "years","months","days","weekdays","holiday_flg","satsun_flg", "special_flg"]
time_df.head(10)

In [ ]:
End_Start_year_list = [
    #年末年始
    "2017-01-02",
    "2017-12-30","2017-12-31",
    "2018-01-01","2018-01-02","2018-01-03","2018-01-04","2018-01-05","2018-01-06","2018-01-07",
    "2018-12-29","2018-12-30","2018-12-31"
]

GW_list = [
    #GW
    "2017-04-29","2017-04-30",
    "2017-05-01","2017-05-02","2017-05-03","2017-05-04","2017-05-05","2017-05-06","2017-05-07",
    "2018-04-27","2018-04-28","2018-04-29","2018-04-30",
    "2018-05-01","2018-05-02","2018-05-03","2018-05-04","2018-05-05","2018-05-06"
    ]

#多分、ハロウィーンまでの残り時間の逆数みたいな変数を作ったほうがいい気がする
Halloween_list = [
    #ハロウィン
    "2017-10-31",
    "2018-10-31",
]

#多分、クリスマスをピークに前30日後１４日で値が入るような変数を作ったほうがいい気がする
Christmas_list = [
#クリスマス
"2017-12-24","2017-12-25",
"2018-12-24","2018-12-25",
]

#多分、バレンタインをピークに前30日後１４日で値が入るような変数を作ったほうがいい気がする
Valentine_list = [
    "2017-02-14",
    "2018-02-14",
]

#多分、ホワイトデーまでの残り時間の逆数みたいな変数を作ったほうがいい気がする
Whiteday_list = [
    "2017-03-14",
    "2018-03-14",
]

In [ ]:
time_df['End_Start_year_flg'] = date.isin(End_Start_year_list).astype("int")
time_df['GW_flg'] = date.isin(GW_list).astype("int")
time_df['Halloween_flg'] = date.isin(Halloween_list).astype("int")
time_df['Christmas_flg'] = date.isin(Christmas_list).astype("int")
time_df['Valentine_flg'] = date.isin(Valentine_list).astype("int")
time_df['Whiteday_flg'] = date.isin(Whiteday_list).astype("int")

In [ ]:
time_df

In [ ]:
time_df.to_pickle(f"/mnt/batch/tasks/shared/LS_root/mounts/clusters/sinchir02/code/Users/td017/kaggle-pipeline/tmp/{CASE}_time_df.pkl")

# numeric

In [ ]:
def aggr_numeric(train,test,cols,key_col,agg_list):
    train_after = train.copy()
    test_after = test.copy()
    
    for col in cols:
        for agg_type in agg_list:
            new_col_name = key_col + "_" + col +"_"+ agg_type
            temp_df =  purchase_merge.groupby(key_col)[col].agg([agg_type]).reset_index().rename(columns={agg_type : new_col_name})
            temp_df.index = list(temp_df[key_col])
            temp_df = temp_df[new_col_name].to_dict()   
            
            train_after[new_col_name] = train[key_col].map(temp_df)
            test_after[new_col_name]  = test[key_col].map(temp_df)
    
    return train_after,test_after

In [ ]:
key_col = "purchase_id"
cols = ["amount","total","p_time"]
agg_list = ['sum','max','min','mean','std','median']

train_tmp2,test_tmp2 = aggr_numeric(train_tmp,test_tmp,cols,key_col,agg_list)

In [ ]:
train_tmp2.head()

In [ ]:
key_col = "mpno"

train_tmp3,test_tmp3 = aggr_numeric(train_tmp2,test_tmp2,cols,key_col,agg_list)

In [ ]:
train_tmp3.head()

In [ ]:
key_col = "mstr"

train_tmp4,test_tmp4 = aggr_numeric(train_tmp3,test_tmp3,cols,key_col,agg_list)

In [ ]:
train_tmp4.head()

In [ ]:
train_tmp4.to_pickle(f"/mnt/batch/tasks/shared/LS_root/mounts/clusters/sinchir02/code/Users/td017/kaggle-pipeline/tmp/{CASE}_train.pkl")
test_tmp4.to_pickle(f"/mnt/batch/tasks/shared/LS_root/mounts/clusters/sinchir02/code/Users/td017/kaggle-pipeline/tmp/{CASE}_test.pkl")

# category

In [ ]:
train_tmp4 = train_after.copy()
test_tmp4 = test_after.copy()

In [ ]:
def aggr_category(train,test,cols,key_col,agg_list):
    from scipy.stats import mode
    def mod(arr):
        print("lol")
        print(mode(arr))
        return mode(arr)[0][0]   
    
    train_after = train.copy()
    test_after = test.copy()
    
    for col in cols:
        for agg_type in agg_list:
            new_col_name = key_col + "_" + col +"_"+ agg_type
            if agg_type == "mode":
                temp_df =  purchase_merge.groupby(key_col)[col].agg(mod).reset_index().rename(columns={col : new_col_name})
            elif agg_type == "nunique":
                temp_df =  purchase_merge.groupby(key_col)[col].nunique().reset_index().rename(columns={col : new_col_name})
            else:
                temp_df =  purchase_merge.groupby(key_col)[col].agg([agg_type]).reset_index().rename(columns={agg_type : new_col_name})
            temp_df.index = list(temp_df[key_col])
            temp_df = temp_df[new_col_name].to_dict()   
            
            train_after[new_col_name] = train[key_col].map(temp_df)
            test_after[new_col_name]  = test[key_col].map(temp_df)
    
    return train_after,test_after

In [ ]:
key_col = "purchase_id"
cols = ['ccl_category_cd4',
        'ccl_jan','jan_name',
        'ccl_category_cd1',
        'ccl_category_cd2',
        'ccl_category_cd3',
       'ccl_category_name1',
       'ccl_category_name2',
       'ccl_category_name3',
       'ccl_category_name4',
        'mpno',
        'mstr'
       ]
agg_list = ['mode','nunique']

train_tmp5,test_tmp5 = aggr_category(train_tmp4,test_tmp4,cols,key_col,agg_list)

In [ ]:
key_col = "mpno"
cols.remove(key_col)
# cols = ["ccl_category_cd4"]
# agg_list = ['mode']

train_tmp6,test_tmp6 = aggr_category(train_tmp5,test_tmp5,cols,key_col,agg_list)

In [ ]:
key_col = "mstr"
cols.remove(key_col)
# cols = ["ccl_category_cd4"]
# agg_list = ['mode']

train_tmp7,test_tmp7 = aggr_category(train_tmp6,test_tmp6,cols,key_col,agg_list)

In [ ]:
# train_after.to_csv(
#     "/mnt/batch/tasks/shared/LS_root/mounts/clusters/sinchir0/code/Users/td017/kaggle-pipeline/input/train_after.csv",
#     index=False
# )

# test_after.to_csv(
#     "/mnt/batch/tasks/shared/LS_root/mounts/clusters/sinchir0/code/Users/td017/kaggle-pipeline/input/test_after.csv",
#     index=False
# )

In [ ]:
train_tmp7.to_pickle(f"/mnt/batch/tasks/shared/LS_root/mounts/clusters/sinchir02/code/Users/td017/kaggle-pipeline/tmp/{CASE}_train.pkl")
test_tmp7.to_pickle(f"/mnt/batch/tasks/shared/LS_root/mounts/clusters/sinchir02/code/Users/td017/kaggle-pipeline/tmp/{CASE}_test.pkl")

# Load_and_pickle_Model

In [ ]:
train_tmp7 = pd.read_pickle(f"/mnt/batch/tasks/shared/LS_root/mounts/clusters/sinchir02/code/Users/td017/kaggle-pipeline/tmp/case8_train.pkl")
test_tmp7 = pd.read_pickle(f"/mnt/batch/tasks/shared/LS_root/mounts/clusters/sinchir02/code/Users/td017/kaggle-pipeline/tmp/case8_test.pkl")

In [ ]:
time_df = pd.read_pickle(f"/mnt/batch/tasks/shared/LS_root/mounts/clusters/sinchir02/code/Users/td017/kaggle-pipeline/tmp/{CASE}_time_df.pkl")

In [ ]:
# train_tmp7['p_date'] = pd.to_datetime(train_tmp7['p_date'] )
# train_tmp8 = pd.merge(train_tmp7,time_df,on='p_date')

# test_tmp7['p_date'] = pd.to_datetime(test_tmp7['p_date'] )
# test_tmp8 = pd.merge(test_tmp7,time_df,on='p_date')

In [ ]:
train_tmp7['p_date'] = pd.to_datetime(train_tmp7['p_date'] )
train_tmp8 = pd.merge(train_tmp7,time_df[["p_date","Christmas_flg"]],on='p_date')

test_tmp7['p_date'] = pd.to_datetime(test_tmp7['p_date'] )
test_tmp8 = pd.merge(test_tmp7,time_df[["p_date","Christmas_flg"]],on='p_date')

In [ ]:
train_tmp8.head()

In [ ]:
test_tmp8.head()

In [ ]:
train_tmp8.to_pickle(f"/mnt/batch/tasks/shared/LS_root/mounts/clusters/sinchir02/code/Users/td017/kaggle-pipeline/tmp/{CASE}_train.pkl")
test_tmp8.to_pickle(f"/mnt/batch/tasks/shared/LS_root/mounts/clusters/sinchir02/code/Users/td017/kaggle-pipeline/tmp/{CASE}_test.pkl")

# TIME_AGG

In [ ]:
train_tmp8 = pd.read_pickle(f"/mnt/batch/tasks/shared/LS_root/mounts/clusters/sinchir02/code/Users/td017/kaggle-pipeline/tmp/{CASE}_train.pkl")
test_tmp8 = pd.read_pickle(f"/mnt/batch/tasks/shared/LS_root/mounts/clusters/sinchir02/code/Users/td017/kaggle-pipeline/tmp/{CASE}_test.pkl")

In [ ]:
train_tmp8.head()

In [ ]:
purchase_merge = pd.read_pickle("/mnt/batch/tasks/shared/LS_root/mounts/clusters/sinchir02/code/Users/td017/kaggle-pipeline/tmp/purchase_merge.csv")

In [ ]:
time_df = pd.read_pickle(f"/mnt/batch/tasks/shared/LS_root/mounts/clusters/sinchir02/code/Users/td017/kaggle-pipeline/tmp/case9_time_df.pkl")

In [ ]:
purchase_merge.head()

In [ ]:
time_df.head()

In [ ]:
purchase_merge['p_date'] = pd.to_datetime(purchase_merge['p_date'])

purchase_merge = purchase_merge.merge(time_df,on='p_date')

In [ ]:
purchase_merge.head()

### numeric

In [ ]:
key_col = "months"
cols = ["amount","total","p_time"]
agg_list = ['sum','max','min','mean','std','median']

train_tmp9,test_tmp9 = aggr_numeric(train_tmp8,test_tmp8,cols,key_col,agg_list)

In [ ]:
key_col = "weekdays"

train_tmp10,test_tmp10 = aggr_numeric(train_tmp9,test_tmp9,cols,key_col,agg_list)

In [ ]:
key_col = "days"

train_tmp11,test_tmp11 = aggr_numeric(train_tmp10,test_tmp10,cols,key_col,agg_list)

In [ ]:
key_col = "holiday_flg"

train_tmp12,test_tmp12 = aggr_numeric(train_tmp11,test_tmp11,cols,key_col,agg_list)

In [ ]:
train_tmp12.to_pickle(f"/mnt/batch/tasks/shared/LS_root/mounts/clusters/sinchir02/code/Users/td017/kaggle-pipeline/tmp/{CASE}_train.pkl")
test_tmp12.to_pickle(f"/mnt/batch/tasks/shared/LS_root/mounts/clusters/sinchir02/code/Users/td017/kaggle-pipeline/tmp/{CASE}_test.pkl")

### category

In [ ]:
# key_col = "months"
# cols = [
#     #'ccl_category_cd4',
#     #'ccl_jan',
#     'jan_name',
#     #'ccl_category_cd1',
#     #'ccl_category_cd2',
#     #'ccl_category_cd3',
#     'ccl_category_name1',
#     'ccl_category_name2',
#     'ccl_category_name3',
#     'ccl_category_name4',
#     #'mpno',
#     #'mstr'
# ]
# agg_list = ['mode','nunique']

# train_tmp13,test_tmp13 = aggr_category(train_tmp12,test_tmp12,cols,key_col,agg_list)

In [ ]:
# key_col = "weekdays"

# train_tmp14,test_tmp14 = aggr_category(train_tmp13,test_tmp13,cols,key_col,agg_list)

In [ ]:
# key_col = "days"

# train_tmp15,test_tmp15 = aggr_category(train_tmp14,test_tmp14,cols,key_col,agg_list)

In [ ]:
# key_col = "holiday_flg"

# train_tmp16,test_tmp16 = aggr_category(train_tmp15,test_tmp15,cols,key_col,agg_list)

In [ ]:
# train_tmp16.to_pickle(f"/mnt/batch/tasks/shared/LS_root/mounts/clusters/sinchir0/code/Users/td017/kaggle-pipeline/tmp/{CASE}_train.pkl")
# test_tmp16.to_pickle(f"/mnt/batch/tasks/shared/LS_root/mounts/clusters/sinchir0/code/Users/td017/kaggle-pipeline/tmp/{CASE}_test.pkl")

# Model

In [10]:
train_after = pd.read_pickle(f"/mnt/batch/tasks/shared/LS_root/mounts/clusters/sinchir02/code/Users/td017/kaggle-pipeline/tmp/case9_train.pkl")
test_after = pd.read_pickle(f"/mnt/batch/tasks/shared/LS_root/mounts/clusters/sinchir02/code/Users/td017/kaggle-pipeline/tmp/case9_test.pkl")

In [11]:
train_tmp = train_after.copy()
test_tmp = test_after.copy()

In [12]:
month_target_per_min_max = pd.read_pickle(f"/mnt/batch/tasks/shared/LS_root/mounts/clusters/sinchir02/code/Users/td017/kaggle-pipeline/tmp/month_per_min_max_df.pkl").add_suffix("_per_months")
day_target_per_min_max = pd.read_pickle(f"/mnt/batch/tasks/shared/LS_root/mounts/clusters/sinchir02/code/Users/td017/kaggle-pipeline/tmp/day_per_min_max_df.pkl").add_suffix("_per_days")

In [13]:
month_target_per_min_max.head()

,months_per_months,スナック_per_months,チョコレート_per_months,RTD_per_months,コーヒードリンク_per_months,米菓_per_months,新ジャンル_per_months,日本茶・麦茶ドリンク_per_months,ビール_per_months,発泡酒_per_months,チューインガム_per_months,水_per_months,その他茶ドリンク_per_months,炭酸水_per_months
0,1,0.638197,0.556225,0.099330,0.031246,0.430260,0.000000,0.041853,0.000000,0.000000,0.125129,0.018580,0.000000,0.078538
1,2,0.754691,1.000000,0.255688,0.000000,0.886637,0.517630,0.000000,0.189315,0.175762,0.157159,0.077859,0.222124,0.000000
2,3,0.626955,0.506733,0.404150,0.088989,1.000000,0.708028,0.431775,0.575860,0.212208,0.378681,0.000000,0.165016,0.091367
3,4,0.876234,0.453102,0.367034,0.278157,0.410407,0.711879,0.325582,0.873317,0.868953,0.557202,0.072100,0.195202,0.344214
4,5,0.078425,0.243080,0.523541,0.522735,0.387490,0.904890,0.538690,0.884929,1.000000,0.000000,0.396753,0.730982,1.000000


In [14]:
train_tmp = pd.merge(train_tmp,month_target_per_min_max, left_on = "months",right_on="months_per_months")
test_tmp = pd.merge(test_tmp,month_target_per_min_max, left_on = "months",right_on="months_per_months")

train_tmp = pd.merge(train_tmp,day_target_per_min_max, left_on = "days",right_on="days_per_days")
test_tmp = pd.merge(test_tmp,day_target_per_min_max, left_on = "days", right_on="days_per_days")

In [15]:
mstr_target_per_min_max = pd.read_pickle(f"/mnt/batch/tasks/shared/LS_root/mounts/clusters/sinchir02/code/Users/td017/kaggle-pipeline/tmp/case13_mstr_per.pkl").add_suffix("_per_mstr").reset_index().rename(columns={"index":"mstr"})

In [16]:
train_tmp = pd.merge(train_tmp,mstr_target_per_min_max, on="mstr")
test_tmp = pd.merge(test_tmp,mstr_target_per_min_max, on="mstr")

In [17]:
print(train_tmp.shape)
print(test_tmp.shape)

(128376, 260)
(96505, 247)


In [18]:
# 気象庁

In [19]:
kisyoutyou = pd.read_csv(f"/mnt/batch/tasks/shared/LS_root/mounts/clusters/sinchir02/code/Users/td017/kaggle-pipeline/tmp/kisyoutyou_2.csv")

In [20]:
kisyoutyou['p_date'] = pd.to_datetime(kisyoutyou['p_date'])
train_tmp = pd.merge(train_tmp,kisyoutyou, on="p_date")
test_tmp = pd.merge(test_tmp,kisyoutyou, on="p_date")

In [21]:
train_after = train_tmp.copy()
test_after = test_tmp.copy()

In [22]:
print(train_after.shape)
print(test_after.shape)

(128376, 266)
(96505, 253)


In [34]:
from pykakasi import kakasi

kakasi = kakasi()

kakasi.setMode('H', 'a')
kakasi.setMode('K', 'a')
kakasi.setMode('J', 'a')

conv = kakasi.getConverter()

In [35]:
columns_list_train = []

for i in train_after.columns.tolist():
    columns_list_train.append(conv.do(i))

train_after.columns = columns_list_train

columns_list_test = []

for i in test_after.columns.tolist():
    columns_list_test.append(conv.do(i))

test_after.columns = columns_list_test

In [ ]:
train_after.head()

In [ ]:
test_after.head()

In [36]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb

class Base_Model(object):
    
    def __init__(self, train_df, test_df, features, target,categoricals=[], n_splits=5, verbose=True):
        self.train_df = train_df
        self.test_df = test_df
        self.features = features
        self.n_splits = n_splits
        self.categoricals = categoricals
        self.target = target
        self.cv = self.get_cv()
        self.verbose = verbose
        self.params = self.get_params()
        self.y_pred, self.score, self.model,self.oof_pred = self.fit()
        
    def train_model(self, train_set, val_set):
        raise NotImplementedError
        
    def get_cv(self):
        cv = StratifiedKFold(n_splits=self.n_splits, shuffle=True, random_state=42)
        return cv.split(self.train_df, self.train_df[self.target])
    
    def get_params(self):
        raise NotImplementedError
        
    def convert_dataset(self, x_train, y_train, x_val, y_val):
        raise NotImplementedError
        
    def convert_x(self, x):
        return x
        
    def fit(self):
        oof_pred = np.zeros((len(self.train_df), ))
        y_pred = np.zeros((len(self.test_df), ))
        for fold, (train_idx, val_idx) in enumerate(self.cv):
            x_train, x_val = self.train_df[self.features].iloc[train_idx], self.train_df[self.features].iloc[val_idx]
            y_train, y_val = self.train_df[self.target][train_idx], self.train_df[self.target][val_idx]
            train_set, val_set = self.convert_dataset(x_train, y_train, x_val, y_val)
            model = self.train_model(train_set, val_set)
            conv_x_val = self.convert_x(x_val)
            oof_pred[val_idx] = model.predict(conv_x_val).reshape(oof_pred[val_idx].shape)
            x_test = self.convert_x(self.test_df[self.features])
            y_pred += model.predict(x_test).reshape(y_pred.shape) / self.n_splits
            print('Partial score of fold {} is: {}'.format(fold, roc_auc_score(y_val, oof_pred[val_idx])))
            score = roc_auc_score(self.train_df[self.target], oof_pred)
        if self.verbose:
            print('AUC score is: ', score)
        return y_pred, score, model, oof_pred

In [37]:
class Lgb_Model(Base_Model):
    def train_model(self, train_set, val_set):
        verbosity = 100 if self.verbose else 0
        return lgb.train(self.params, train_set, valid_sets=[train_set, val_set], verbose_eval=verbosity)
        
    def convert_dataset(self, x_train, y_train, x_val, y_val):
        train_set = lgb.Dataset(x_train, y_train, categorical_feature=self.categoricals)
        val_set = lgb.Dataset(x_val, y_val, categorical_feature=self.categoricals)
        return train_set, val_set
        
    def get_params(self):
        params = {'n_estimators':5000,
                    'boosting_type': 'gbdt',
                    'objective': 'binary',
                    'metric': 'auc',
                    'subsample': 0.75,
                    'subsample_freq': 1,
                    'learning_rate': 0.05,
                    'feature_fraction': 0.9,
                    'max_depth': 7,
                    'lambda_l1': 1,  
                    'lambda_l2': 1,
                    'early_stopping_rounds': 100
                    }
        return params

In [38]:
target_list = [ 
 '130123',
 '130125',
 '130129',
 '130131',
 '140307',
 '140313',
 '140316',
 '140317',
 '140321',
 '140501',
 '140505',
 '140641',
 '140691'
]

In [39]:
remove_list = [
    'purchase_id',
    'years',
    'p_date'
]

In [40]:
features = train_after.columns.tolist()

for i in target_list:
    features.remove(i)
    
for i in remove_list:
    features.remove(i)

In [41]:
features

['mpno',
 'mstr',
 'p_time',
 'purchase_id_amount_sum',
 'purchase_id_amount_max',
 'purchase_id_amount_min',
 'purchase_id_amount_mean',
 'purchase_id_amount_std',
 'purchase_id_amount_median',
 'purchase_id_total_sum',
 'purchase_id_total_max',
 'purchase_id_total_min',
 'purchase_id_total_mean',
 'purchase_id_total_std',
 'purchase_id_total_median',
 'purchase_id_p_time_sum',
 'purchase_id_p_time_max',
 'purchase_id_p_time_min',
 'purchase_id_p_time_mean',
 'purchase_id_p_time_std',
 'purchase_id_p_time_median',
 'mpno_amount_sum',
 'mpno_amount_max',
 'mpno_amount_min',
 'mpno_amount_mean',
 'mpno_amount_std',
 'mpno_amount_median',
 'mpno_total_sum',
 'mpno_total_max',
 'mpno_total_min',
 'mpno_total_mean',
 'mpno_total_std',
 'mpno_total_median',
 'mpno_p_time_sum',
 'mpno_p_time_max',
 'mpno_p_time_min',
 'mpno_p_time_mean',
 'mpno_p_time_std',
 'mpno_p_time_median',
 'mstr_amount_sum',
 'mstr_amount_max',
 'mstr_amount_min',
 'mstr_amount_mean',
 'mstr_amount_std',
 'mstr_amoun

In [42]:
#Label Enc
def LabelEnc(train,test):
    train_after = train.copy()
    test_after = test.copy()
    
    train["type"] = "train"
    test["type"] = "test"
    
    # データを全結合
    all_tmp = pd.concat([train,test],sort=False)
    
    # Labelencを行うリストを取得
    obj_lost = all_tmp.select_dtypes("O").columns.tolist()
    obj_lost.remove("type")
    
    ### LabelEncを実施
    import category_encoders as ce
    #ce_oe = ce.OrdinalEncoder(cols=obj_lost,handle_unknown='impute')
    ce_oe = ce.OrdinalEncoder(cols=obj_lost)
    all_tmp_2 = ce_oe.fit_transform(all_tmp)
    all_tmp_2_train = all_tmp_2[all_tmp_2["type"] == "train"]
    all_tmp_2_test = all_tmp_2[all_tmp_2["type"] == "test"]
    #文字を序数に変換
    for i in obj_lost:
        train_after[i] = all_tmp_2_train[i] - 1
        test_after[i] = all_tmp_2_test[i] - 1
    
    return train_after,test_after

train_after,test_after = LabelEnc(train_after,test_after)

In [43]:
print("################### 1 ###################")
lgb_model_130123 = Lgb_Model(train_after, test_after, features, '130123')
print("################### 2 ###################")
lgb_model_130125 = Lgb_Model(train_after, test_after, features, '130125')
print("################### 3 ###################")
lgb_model_130129 = Lgb_Model(train_after, test_after, features, '130129')
print("################### 4 ###################")
lgb_model_130131 = Lgb_Model(train_after, test_after, features, '130131')
print("################### 5 ###################")
lgb_model_140307 = Lgb_Model(train_after, test_after, features, '140307')
print("################### 6 ###################")
lgb_model_140313 = Lgb_Model(train_after, test_after, features, '140313')
print("################### 7 ###################")
lgb_model_140316 = Lgb_Model(train_after, test_after, features, '140316')
print("################### 8 ###################")
lgb_model_140317 = Lgb_Model(train_after, test_after, features, '140317')
print("################### 9 ###################")
lgb_model_140321 = Lgb_Model(train_after, test_after, features, '140321')
print("################### 10 ###################")
lgb_model_140501 = Lgb_Model(train_after, test_after, features, '140501')
print("################### 11 ###################")
lgb_model_140505 = Lgb_Model(train_after, test_after, features, '140505')
print("################### 12 ###################")
lgb_model_140641 = Lgb_Model(train_after, test_after, features, '140641')
print("################### 13 ###################")
lgb_model_140691 = Lgb_Model(train_after, test_after, features, '140691')

################### 1 ###################


/anaconda/envs/azureml_py36/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/lightgbm/basic.py:1243: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.771811	valid_1's auc: 0.732927
[200]	training's auc: 0.810026	valid_1's auc: 0.746359
[300]	training's auc: 0.838041	valid_1's auc: 0.749858
[400]	training's auc: 0.860632	valid_1's auc: 0.751393
[500]	training's auc: 0.880201	valid_1's auc: 0.753106
[600]	training's auc: 0.896402	valid_1's auc: 0.752851
Early stopping, best iteration is:
[531]	training's auc: 0.885487	valid_1's auc: 0.753323
Partial score of fold 0 is: 0.7533234808346417


/anaconda/envs/azureml_py36/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/lightgbm/basic.py:1243: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.771633	valid_1's auc: 0.73243
[200]	training's auc: 0.810016	valid_1's auc: 0.743455
[300]	training's auc: 0.838145	valid_1's auc: 0.747615
[400]	training's auc: 0.860852	valid_1's auc: 0.749261
[500]	training's auc: 0.880081	valid_1's auc: 0.74943
Early stopping, best iteration is:
[483]	training's auc: 0.877024	valid_1's auc: 0.749879
Partial score of fold 1 is: 0.7498790527943615


...

Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.968666	valid_1's auc: 0.938447
[200]	training's auc: 0.984838	valid_1's auc: 0.944198
[300]	training's auc: 0.991641	valid_1's auc: 0.945354
Early stopping, best iteration is:
[291]	training's auc: 0.991218	valid_1's auc: 0.945428
Partial score of fold 2 is: 0.9454280511744414


/anaconda/envs/azureml_py36/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/lightgbm/basic.py:1243: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.96648	valid_1's auc: 0.935458
[200]	training's auc: 0.983731	valid_1's auc: 0.943851
[300]	training's auc: 0.991491	valid_1's auc: 0.945353
[400]	training's auc: 0.995398	valid_1's auc: 0.945892
Early stopping, best iteration is:
[345]	training's auc: 0.993602	valid_1's auc: 0.946244
Partial score of fold 3 is: 0.9462436667686034


/anaconda/envs/azureml_py36/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/lightgbm/basic.py:1243: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.970521	valid_1's auc: 0.912895
[200]	training's auc: 0.985466	valid_1's auc: 0.920643
[300]	training's auc: 0.992133	valid_1's auc: 0.92348
[400]	training's auc: 0.995579	valid_1's auc: 0.926247
[500]	training's auc: 0.997681	valid_1's auc: 0.927421
[600]	training's auc: 0.998872	valid_1's auc: 0.92846
[700]	training's auc: 0.99948	valid_1's auc: 0.928595
[800]	training's auc: 0.999784	valid_1's auc: 0.929487
[900]	training's auc: 0.999922	valid_1's auc: 0.9292
Early stopping, best iteration is:
[837]	training's auc: 0.999852	valid_1's auc: 0.929872
Partial score of fold 4 is: 0.9298715171555684
AUC score is:  0.9420280468776825


In [44]:
# print(f"130123_AUC: {lgb_model_130123.score}")
# print(f"130125_AUC: {lgb_model_130125.score}")
# print(f"130129_AUC: {lgb_model_130129.score}")
# print(f"130131_AUC: {lgb_model_130131.score}")
# print(f"140307_AUC: {lgb_model_140307.score}")
# print(f"140313_AUC: {lgb_model_140313.score}")
# print(f"140316_AUC: {lgb_model_140316.score}")
# print(f"140317_AUC: {lgb_model_140317.score}")
# print(f"140321_AUC: {lgb_model_140321.score}")
# print(f"140501_AUC: {lgb_model_140501.score}")
# print(f"140505_AUC: {lgb_model_140505.score}")
# print(f"140641_AUC: {lgb_model_140641.score}")
# print(f"140691_AUC: {lgb_model_140691.score}")

In [45]:
oof_pred = pd.DataFrame({
   "130123" :  lgb_model_130123.oof_pred,
   "130125" :  lgb_model_130125.oof_pred,
   "130129" :  lgb_model_130129.oof_pred,
   "130131" :  lgb_model_130131.oof_pred,
   "140307" :  lgb_model_140307.oof_pred,
   "140313" :  lgb_model_140313.oof_pred,
   "140316" :  lgb_model_140316.oof_pred,
   "140317" :  lgb_model_140317.oof_pred,
   "140321" :  lgb_model_140321.oof_pred,
   "140501" :  lgb_model_140501.oof_pred,
   "140505" :  lgb_model_140505.oof_pred,
   "140641" :  lgb_model_140641.oof_pred,
   "140691" :  lgb_model_140691.oof_pred,
 })

oof_pred.to_pickle(f"/mnt/batch/tasks/shared/LS_root/mounts/clusters/sinchir02/code/Users/td017/kaggle-pipeline/tmp/{CASE}_oof_pred.pkl")

In [4]:
from utils import *
config = config_read()

ModuleNotFoundError: No module named 'utils'

In [2]:
CASE = "case1"
path_w = f"{CASE}"

In [3]:
path_w

'case1'

In [46]:
path_w = f'/mnt/batch/tasks/shared/LS_root/mounts/clusters/sinchir02/code/Users/td017/kaggle-pipeline/model/{CASE}_lgb_score.txt'

s1 = f"130123_AUC: {lgb_model_130123.score}"
s2 = f"130125_AUC: {lgb_model_130125.score}"
s3 = f"130129_AUC: {lgb_model_130129.score}"
s4 = f"130131_AUC: {lgb_model_130131.score}"
s5 = f"140307_AUC: {lgb_model_140307.score}"
s6 = f"140313_AUC: {lgb_model_140313.score}"
s7 = f"140316_AUC: {lgb_model_140316.score}"
s8 = f"140317_AUC: {lgb_model_140317.score}"
s9 = f"140321_AUC: {lgb_model_140321.score}"
s10 = f"140501_AUC: {lgb_model_140501.score}"
s11 = f"140505_AUC: {lgb_model_140505.score}"
s12 = f"140641_AUC: {lgb_model_140641.score}"
s13 = f"140691_AUC: {lgb_model_140691.score}"

with open(path_w, mode='w') as f:
    f.write(s1+"\n")
    f.write(s2+"\n")
    f.write(s3+"\n")
    f.write(s4+"\n")
    f.write(s5+"\n")
    f.write(s6+"\n")
    f.write(s7+"\n")
    f.write(s8+"\n")
    f.write(s9+"\n")
    f.write(s10+"\n")
    f.write(s11+"\n")
    f.write(s12+"\n")
    f.write(s13+"\n")

with open(path_w) as f:
    print(f.read())

130123_AUC: 0.7516361499218029
130125_AUC: 0.8143444472271061
130129_AUC: 0.7408489989012229
130131_AUC: 0.7555031839744257
140307_AUC: 0.8367614554684554
140313_AUC: 0.8280569385165586
140316_AUC: 0.8148541893826009
140317_AUC: 0.8418859569958494
140321_AUC: 0.8757060088536831
140501_AUC: 0.91238622058242
140505_AUC: 0.8902325875931045
140641_AUC: 0.8765328184220949
140691_AUC: 0.9420280468776825



In [47]:
sample_submission["130123"] = lgb_model_130123.y_pred
sample_submission["130125"] = lgb_model_130125.y_pred
sample_submission["130129"] = lgb_model_130129.y_pred
sample_submission["130131"] = lgb_model_130131.y_pred
sample_submission["140307"] = lgb_model_140307.y_pred
sample_submission["140313"] = lgb_model_140313.y_pred
sample_submission["140316"] = lgb_model_140316.y_pred
sample_submission["140317"] = lgb_model_140317.y_pred
sample_submission["140321"] = lgb_model_140321.y_pred
sample_submission["140501"] = lgb_model_140501.y_pred
sample_submission["140505"] = lgb_model_140505.y_pred
sample_submission["140641"] = lgb_model_140641.y_pred
sample_submission["140691"] = lgb_model_140691.y_pred

In [48]:
sample_submission.head()

,130123,130125,130129,130131,140307,140313,140316,140317,140321,140501,140505,140641,140691
0,0.040232,0.006860,0.025696,0.067452,0.032766,0.009906,0.004553,0.008393,0.009025,0.018505,0.030068,0.025426,0.007609
1,0.076402,0.013645,0.045617,0.084973,0.040603,0.011788,0.010483,0.005316,0.005111,0.254897,0.153382,0.037649,0.003275
2,0.241824,0.030543,0.132799,0.228855,0.078287,0.054810,0.008997,0.007156,0.008335,0.028201,0.031096,0.022021,0.004850
3,0.109612,0.009601,0.064301,0.294879,0.040440,0.044881,0.010482,0.011213,0.015895,0.030360,0.030806,0.018678,0.006798
4,0.148123,0.009579,0.065051,0.177052,0.119795,0.041511,0.011092,0.006895,0.010336,0.011060,0.061648,0.048735,0.036030


In [49]:
sample_submission.shape

(96505, 13)

In [51]:
SAME_PATH

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/sinchir02/code/'

In [50]:
sample_submission.to_csv(f"{SAME_PATH}" + "/Users/td017/kaggle-pipeline/submission/sub_" + f"{CASE}" + ".csv",index=False)

# XGBoost

In [ ]:
# class Xgb_Model(Base_Model):
    
#     def train_model(self, train_set, val_set):
#         verbosity = 100 if self.verbose else 0
#         return xgb.train(self.params, train_set, 
#                          num_boost_round=5000, evals=[(train_set, 'train'), (val_set, 'val')], 
#                          verbose_eval=verbosity, early_stopping_rounds=100)
        
#     def convert_dataset(self, x_train, y_train, x_val, y_val):
#         train_set = xgb.DMatrix(x_train, y_train)
#         val_set = xgb.DMatrix(x_val, y_val)
#         return train_set, val_set
    
#     def convert_x(self, x):
#         return xgb.DMatrix(x)
        
#     def get_params(self):
#         params = {'objective': 'binary:logistic',
#          'learning_rate': 0.01,
#         'eval_metric': 'logloss',
#         #'num_class': 9,
#         'max_depth': 12,
#         'eta': 0.1,
#         'min_child_weight': 32,
#         'subsample': 0.9,
#         'colsample_bytree': 0.8,
#         'silent': 1,
#         'random_state': 71,
#         #'num_round': 10000,
#         'early_stopping_rounds': 10}
        
#         #{'colsample_bytree': 0.8,                 
#             #'learning_rate': 0.01,
#             #'max_depth': 7,#10
# #             'subsample': 1,
# #             'objective':'reg:squarederror',
# #             #'eval_metric':'rmse',
# #             'min_child_weight':32,#16,
# #             'gamma':0.25,
# #             'n_estimators':5000}

#         return params

In [ ]:
# print("################### 1 ###################")
# xgb_model_130123 = Xgb_Model(train_after, test_after, features, '130123')
# print("################### 2 ###################")
# xgb_model_130125 = Xgb_Model(train_after, test_after, features, '130125')
# print("################### 3 ###################")
# xgb_model_130129 = Xgb_Model(train_after, test_after, features, '130129')
# print("################### 4 ###################")
# xgb_model_130131 = Xgb_Model(train_after, test_after, features, '130131')
# print("################### 5 ###################")
# xgb_model_140307 = Xgb_Model(train_after, test_after, features, '140307')
# print("################### 6 ###################")
# xgb_model_140313 = Xgb_Model(train_after, test_after, features, '140313')
# print("################### 7 ###################")
# xgb_model_140316 = Xgb_Model(train_after, test_after, features, '140316')
# print("################### 8 ###################")
# xgb_model_140317 = Xgb_Model(train_after, test_after, features, '140317')
# print("################### 9 ###################")
# xgb_model_140321 = Xgb_Model(train_after, test_after, features, '140321')
# print("################### 10 ###################")
# xgb_model_140501 = Xgb_Model(train_after, test_after, features, '140501')
# print("################### 11 ###################")
# xgb_model_140505 = Xgb_Model(train_after, test_after, features, '140505')
# print("################### 12 ###################")
# xgb_model_140641 = Xgb_Model(train_after, test_after, features, '140641')
# print("################### 13 ###################")
# xgb_model_140691 = Xgb_Model(train_after, test_after, features, '140691')

In [ ]:
# print(f"130123_AUC: {xgb_model_130123.score}")
# print(f"130125_AUC: {xgb_model_130125.score}")
# print(f"130129_AUC: {xgb_model_130129.score}")
# print(f"130131_AUC: {xgb_model_130131.score}")
# print(f"140307_AUC: {xgb_model_140307.score}")
# print(f"140313_AUC: {xgb_model_140313.score}")
# print(f"140316_AUC: {xgb_model_140316.score}")
# print(f"140317_AUC: {xgb_model_140317.score}")
# print(f"140321_AUC: {xgb_model_140321.score}")
# print(f"140501_AUC: {xgb_model_140501.score}")
# print(f"140505_AUC: {xgb_model_140505.score}")
# print(f"140641_AUC: {xgb_model_140641.score}")
# print(f"140691_AUC: {xgb_model_140691.score}")

In [ ]:
# path_w = f'/mnt/batch/tasks/shared/LS_root/mounts/clusters/sinchir0/code/Users/td017/kaggle-pipeline/model/{CASE}_xgb_score.txt'

# s1 = f"130123_AUC: {xgb_model_130123.score}"
# s2 = f"130125_AUC: {xgb_model_130125.score}"
# s3 = f"130129_AUC: {xgb_model_130129.score}"
# s4 = f"130131_AUC: {xgb_model_130131.score}"
# s5 = f"140307_AUC: {xgb_model_140307.score}"
# s6 = f"140313_AUC: {xgb_model_140313.score}"
# s7 = f"140316_AUC: {xgb_model_140316.score}"
# s8 = f"140317_AUC: {xgb_model_140317.score}"
# s9 = f"140321_AUC: {xgb_model_140321.score}"
# s10 = f"140501_AUC: {xgb_model_140501.score}"
# s11 = f"140505_AUC: {xgb_model_140505.score}"
# s12 = f"140641_AUC: {xgb_model_140641.score}"
# s13 = f"140691_AUC: {xgb_model_140691.score}"

# with open(path_w, mode='w') as f:
#     f.write(s1+"\n")
#     f.write(s2+"\n")
#     f.write(s3+"\n")
#     f.write(s4+"\n")
#     f.write(s5+"\n")
#     f.write(s6+"\n")
#     f.write(s7+"\n")
#     f.write(s8+"\n")
#     f.write(s9+"\n")
#     f.write(s10+"\n")
#     f.write(s11+"\n")
#     f.write(s12+"\n")
#     f.write(s13+"\n")

# with open(path_w) as f:
#     print(f.read())

# Stackingのような何か

### train

In [52]:
oof_pred_stack = pd.read_pickle(f"/mnt/batch/tasks/shared/LS_root/mounts/clusters/sinchir02/code/Users/td017/kaggle-pipeline/tmp/{CASE}_oof_pred.pkl")

In [53]:
oof_pred_stack

,130123,130125,130129,130131,140307,140313,140316,140317,140321,140501,140505,140641,140691
0,0.040674,0.005444,0.062851,0.042257,0.025029,0.008335,0.001427,0.003847,0.007631,0.003609,0.028405,0.017969,0.007592
1,0.085360,0.008760,0.065909,0.170129,0.029627,0.016038,0.015314,0.002974,0.000512,0.005411,0.023297,0.022189,0.005705
2,0.126139,0.003443,0.127633,0.063036,0.012105,0.027708,0.006339,0.025767,0.000768,0.012390,0.022045,0.049357,0.001946
3,0.121464,0.009970,0.275874,0.093228,0.138492,0.014284,0.002960,0.003669,0.003650,0.031252,0.027514,0.097828,0.006208
4,0.157340,0.009805,0.071833,0.200286,0.175853,0.016206,0.009456,0.003538,0.003391,0.015241,0.034619,0.079691,0.021228
5,0.057167,0.014837,0.077358,0.195276,0.024284,0.017331,0.008168,0.008273,0.001174,0.216117,0.078690,0.043991,0.005783
6,0.122690,0.004273,0.063108,0.262616,0.036614,0.061340,0.008512,0.003397,0.002626,0.195561,0.034565,0.016314,0.005006
7,0.109098,0.005392,0.087265,0.161923,0.336647,0.009870,0.009443,0.002951,0.003167,0.006566,0.024890,0.052241,0.011596
8,0.046405,0.005422,0.041465,0.044197,0.042708,0.027321,0.002224,0.011864,0.002540,0.063945,0.061068,0.070388,0.024996
9,0.365136,0.005485,0.081919,0.094931,0.019410,0.006162,0.013284,0.005044,0.005173,0.008816,0.030763,0.070182,0.005038


In [54]:
oof_pred_stack = oof_pred_stack.rename(columns=lambda s: s+"_pred")

In [55]:
train

,purchase_id,130123,130125,130129,130131,140307,140313,140316,140317,140321,140501,140505,140641,140691
0,njibeyLPrsnu4HCopjBihW,0,0,0,0,0,0,0,0,0,0,0,0,0
1,FqBfZgvrWVNMsCqGmZMdv3,0,0,1,0,0,0,0,0,0,0,0,0,0
2,KYE5JJ4y6zJBipkCKobwVg,0,0,0,0,0,0,0,0,0,0,0,0,0
3,tC5JqjsVxsKxQ8Ykk9S7fg,0,0,0,1,0,0,0,0,0,0,0,0,0
4,Pigjc37smwP2E3Z4VtKinB,0,0,0,0,0,0,0,0,0,0,1,0,0
5,YCHTGgFS6shv3GCMB7sB5j,0,0,0,1,0,0,0,0,0,0,0,0,0
6,nQsjoHBDtiJvKNUxzUoR4d,0,0,0,0,0,0,0,0,0,0,0,0,0
7,LXdStjW5USNHWpcXHd4E5j,0,0,0,0,0,0,1,0,0,0,0,0,0
8,9qw6QFmtqjjbPS9pMrwi7S,0,0,1,0,0,0,0,0,0,0,0,0,0
9,e6qaQuMJ3xsZJ96QmgCnxN,0,0,0,0,0,0,0,0,0,0,0,0,0


In [56]:
train_plus_oof_pred = pd.concat([train,oof_pred_stack],axis=1)

In [57]:
train_plus_oof_pred

,purchase_id,130123,130125,130129,130131,140307,140313,140316,140317,140321,140501,140505,140641,140691,130123_pred,130125_pred,130129_pred,130131_pred,140307_pred,140313_pred,140316_pred,140317_pred,140321_pred,140501_pred,140505_pred,140641_pred,140691_pred
0,njibeyLPrsnu4HCopjBihW,0,0,0,0,0,0,0,0,0,0,0,0,0,0.040674,0.005444,0.062851,0.042257,0.025029,0.008335,0.001427,0.003847,0.007631,0.003609,0.028405,0.017969,0.007592
1,FqBfZgvrWVNMsCqGmZMdv3,0,0,1,0,0,0,0,0,0,0,0,0,0,0.085360,0.008760,0.065909,0.170129,0.029627,0.016038,0.015314,0.002974,0.000512,0.005411,0.023297,0.022189,0.005705
2,KYE5JJ4y6zJBipkCKobwVg,0,0,0,0,0,0,0,0,0,0,0,0,0,0.126139,0.003443,0.127633,0.063036,0.012105,0.027708,0.006339,0.025767,0.000768,0.012390,0.022045,0.049357,0.001946
3,tC5JqjsVxsKxQ8Ykk9S7fg,0,0,0,1,0,0,0,0,0,0,0,0,0,0.121464,0.009970,0.275874,0.093228,0.138492,0.014284,0.002960,0.003669,0.003650,0.031252,0.027514,0.097828,0.006208
4,Pigjc37smwP2E3Z4VtKinB,0,0,0,0,0,0,0,0,0,0,1,0,0,0.157340,0.009805,0.071833,0.200286,0.175853,0.016206,0.009456,0.003538,0.003391,0.015241,0.034619,0.079691,0.021228
5,YCHTGgFS6shv3GCMB7sB5j,0,0,0,1,0,0,0,0,0,0,0,0,0,0.057167,0.014837,0.077358,0.195276,0.024284,0.017331,0.008168,0.008273,0.001174,0.216117,0.078690,0.043991,0.005783
6,nQsjoHBDtiJvKNUxzUoR4d,0,0,0,0,0,0,0,0,0,0,0,0,0,0.122690,0.004273,0.063108,0.262616,0.036614,0.061340,0.008512,0.003397,0.002626,0.195561,0.034565,0.016314,0.005006
7,LXdStjW5USNHWpcXHd4E5j,0,0,0,0,0,0,1,0,0,0,0,0,0,0.109098,0.005392,0.087265,0.161923,0.336647,0.009870,0.009443,0.002951,0.003167,0.006566,0.024890,0.052241,0.011596
8,9qw6QFmtqjjbPS9pMrwi7S,0,0,1,0,0,0,0,0,0,0,0,0,0,0.046405,0.005422,0.041465,0.044197,0.042708,0.027321,0.002224,0.011864,0.002540,0.063945,0.061068,0.070388,0.024996
9,e6qaQuMJ3xsZJ96QmgCnxN,0,0,0,0,0,0,0,0,0,0,0,0,0,0.365136,0.005485,0.081919,0.094931,0.019410,0.006162,0.013284,0.005044,0.005173,0.008816,0.030763,0.070182,0.005038


### test

In [58]:
test

,purchase_id
0,C3rcdjjRyw9qSh6NcZMKSX
1,Y56uzwqQzynHYZ4bDfLPp5
2,xDMdFERmC7CD9yFvyvKJnh
3,zzZENdjz7SvUQkGZV45afF
4,zFWkhHbLYJ9Fh5kUvCrx4g
5,Z9668Qr6T63NGv2vshFZ22
6,iibUNHJdKdwir3YoN23fYB
7,mRUYb9nmdychPJVDTmTuEM
8,Svsd4RkZcFTcnnhHTrtto4
9,2EvrkYaUPmeQxnk6RiVQb7


In [59]:
y_pred = pd.read_csv(f"{SAME_PATH}"+f"Users/td017/kaggle-pipeline/submission/sub_{CASE}.csv")

In [60]:
y_pred

,130123,130125,130129,130131,140307,140313,140316,140317,140321,140501,140505,140641,140691
0,0.040232,0.006860,0.025696,0.067452,0.032766,0.009906,0.004553,0.008393,0.009025,0.018505,0.030068,0.025426,0.007609
1,0.076402,0.013645,0.045617,0.084973,0.040603,0.011788,0.010483,0.005316,0.005111,0.254897,0.153382,0.037649,0.003275
2,0.241824,0.030543,0.132799,0.228855,0.078287,0.054810,0.008997,0.007156,0.008335,0.028201,0.031096,0.022021,0.004850
3,0.109612,0.009601,0.064301,0.294879,0.040440,0.044881,0.010482,0.011213,0.015895,0.030360,0.030806,0.018678,0.006798
4,0.148123,0.009579,0.065051,0.177052,0.119795,0.041511,0.011092,0.006895,0.010336,0.011060,0.061648,0.048735,0.036030
5,0.110663,0.021188,0.080869,0.116072,0.106700,0.028532,0.011348,0.006263,0.029676,0.095509,0.040977,0.013331,0.005558
6,0.157866,0.010783,0.074565,0.174603,0.192951,0.029467,0.011864,0.004071,0.006119,0.010505,0.054553,0.019914,0.006475
7,0.137579,0.035408,0.085052,0.300336,0.137301,0.021337,0.004689,0.007803,0.020294,0.044519,0.060856,0.035409,0.009408
8,0.109786,0.008904,0.089285,0.179042,0.183370,0.035798,0.004544,0.005758,0.021637,0.172807,0.124942,0.056038,0.004552
9,0.107374,0.025556,0.051660,0.219676,0.158542,0.139714,0.013758,0.013936,0.006682,0.020089,0.038128,0.021894,0.003825


In [61]:
y_pred = y_pred.rename(columns=lambda s: s+"_pred")

In [62]:
test_plus_pred = pd.concat([test,y_pred],axis=1)

In [63]:
test_plus_pred.head()

,purchase_id,130123_pred,130125_pred,130129_pred,130131_pred,140307_pred,140313_pred,140316_pred,140317_pred,140321_pred,140501_pred,140505_pred,140641_pred,140691_pred
0,C3rcdjjRyw9qSh6NcZMKSX,0.040232,0.006860,0.025696,0.067452,0.032766,0.009906,0.004553,0.008393,0.009025,0.018505,0.030068,0.025426,0.007609
1,Y56uzwqQzynHYZ4bDfLPp5,0.076402,0.013645,0.045617,0.084973,0.040603,0.011788,0.010483,0.005316,0.005111,0.254897,0.153382,0.037649,0.003275
2,xDMdFERmC7CD9yFvyvKJnh,0.241824,0.030543,0.132799,0.228855,0.078287,0.054810,0.008997,0.007156,0.008335,0.028201,0.031096,0.022021,0.004850
3,zzZENdjz7SvUQkGZV45afF,0.109612,0.009601,0.064301,0.294879,0.040440,0.044881,0.010482,0.011213,0.015895,0.030360,0.030806,0.018678,0.006798
4,zFWkhHbLYJ9Fh5kUvCrx4g,0.148123,0.009579,0.065051,0.177052,0.119795,0.041511,0.011092,0.006895,0.010336,0.011060,0.061648,0.048735,0.036030


In [64]:
features = train_plus_oof_pred.columns.tolist()

for i in target_list:
    features.remove(i)
    
features.remove('purchase_id')

In [65]:
features

['130123_pred',
 '130125_pred',
 '130129_pred',
 '130131_pred',
 '140307_pred',
 '140313_pred',
 '140316_pred',
 '140317_pred',
 '140321_pred',
 '140501_pred',
 '140505_pred',
 '140641_pred',
 '140691_pred']

In [66]:
train_lgb = train_plus_oof_pred.copy()
test_lgb = test_plus_pred.copy()

print("################### 1 ###################")
lgb_model_130123 = Lgb_Model(train_lgb, test_lgb, features, '130123')
print("################### 2 ###################")
lgb_model_130125 = Lgb_Model(train_lgb, test_lgb, features, '130125')
print("################### 3 ###################")
lgb_model_130129 = Lgb_Model(train_lgb, test_lgb, features, '130129')
print("################### 4 ###################")
lgb_model_130131 = Lgb_Model(train_lgb, test_lgb, features, '130131')
print("################### 5 ###################")
lgb_model_140307 = Lgb_Model(train_lgb, test_lgb, features, '140307')
print("################### 6 ###################")
lgb_model_140313 = Lgb_Model(train_lgb, test_lgb, features, '140313')
print("################### 7 ###################")
lgb_model_140316 = Lgb_Model(train_lgb, test_lgb, features, '140316')
print("################### 8 ###################")
lgb_model_140317 = Lgb_Model(train_lgb, test_lgb, features, '140317')
print("################### 9 ###################")
lgb_model_140321 = Lgb_Model(train_lgb, test_lgb, features, '140321')
print("################### 10 ###################")
lgb_model_140501 = Lgb_Model(train_lgb, test_lgb, features, '140501')
print("################### 11 ###################")
lgb_model_140505 = Lgb_Model(train_lgb, test_lgb, features, '140505')
print("################### 12 ###################")
lgb_model_140641 = Lgb_Model(train_lgb, test_lgb, features, '140641')
print("################### 13 ###################")
lgb_model_140691 = Lgb_Model(train_lgb, test_lgb, features, '140691')

################### 1 ###################


/anaconda/envs/azureml_py36/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/lightgbm/basic.py:1243: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.711365	valid_1's auc: 0.490449
Early stopping, best iteration is:
[1]	training's auc: 0.525443	valid_1's auc: 0.504114
Partial score of fold 0 is: 0.504113595358015


/anaconda/envs/azureml_py36/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/lightgbm/basic.py:1243: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.716301	valid_1's auc: 0.494107
Early stopping, best iteration is:
[1]	training's auc: 0.521273	valid_1's auc: 0.500291
Partial score of fold 1 is: 0.5002914793586253


...

Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.867368	valid_1's auc: 0.495368
Early stopping, best iteration is:
[8]	training's auc: 0.660654	valid_1's auc: 0.510554
Partial score of fold 2 is: 0.5105543428037324


/anaconda/envs/azureml_py36/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/lightgbm/basic.py:1243: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.85557	valid_1's auc: 0.492854
Early stopping, best iteration is:
[1]	training's auc: 0.551834	valid_1's auc: 0.515653
Partial score of fold 3 is: 0.5156528211272262


/anaconda/envs/azureml_py36/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/lightgbm/basic.py:1243: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.864942	valid_1's auc: 0.511347
Early stopping, best iteration is:
[6]	training's auc: 0.648858	valid_1's auc: 0.516193
Partial score of fold 4 is: 0.5161926971726096
AUC score is:  0.5095297519169797


In [67]:
# print(f"130123_AUC: {lgb_model_130123.score}")
# print(f"130125_AUC: {lgb_model_130125.score}")
# print(f"130129_AUC: {lgb_model_130129.score}")
# print(f"130131_AUC: {lgb_model_130131.score}")
# print(f"140307_AUC: {lgb_model_140307.score}")
# print(f"140313_AUC: {lgb_model_140313.score}")
# print(f"140316_AUC: {lgb_model_140316.score}")
# print(f"140317_AUC: {lgb_model_140317.score}")
# print(f"140321_AUC: {lgb_model_140321.score}")
# print(f"140501_AUC: {lgb_model_140501.score}")
# print(f"140505_AUC: {lgb_model_140505.score}")
# print(f"140641_AUC: {lgb_model_140641.score}")
# print(f"140691_AUC: {lgb_model_140691.score}")

In [68]:
path_w = f'/mnt/batch/tasks/shared/LS_root/mounts/clusters/sinchir02/code/Users/td017/kaggle-pipeline/model/{CASE}_lgb_score_second.txt'

s1 = f"130123_AUC: {lgb_model_130123.score}"
s2 = f"130125_AUC: {lgb_model_130125.score}"
s3 = f"130129_AUC: {lgb_model_130129.score}"
s4 = f"130131_AUC: {lgb_model_130131.score}"
s5 = f"140307_AUC: {lgb_model_140307.score}"
s6 = f"140313_AUC: {lgb_model_140313.score}"
s7 = f"140316_AUC: {lgb_model_140316.score}"
s8 = f"140317_AUC: {lgb_model_140317.score}"
s9 = f"140321_AUC: {lgb_model_140321.score}"
s10 = f"140501_AUC: {lgb_model_140501.score}"
s11 = f"140505_AUC: {lgb_model_140505.score}"
s12 = f"140641_AUC: {lgb_model_140641.score}"
s13 = f"140691_AUC: {lgb_model_140691.score}"

with open(path_w, mode='w') as f:
    f.write(s1+"\n")
    f.write(s2+"\n")
    f.write(s3+"\n")
    f.write(s4+"\n")
    f.write(s5+"\n")
    f.write(s6+"\n")
    f.write(s7+"\n")
    f.write(s8+"\n")
    f.write(s9+"\n")
    f.write(s10+"\n")
    f.write(s11+"\n")
    f.write(s12+"\n")
    f.write(s13+"\n")

with open(path_w) as f:
    print(f.read())

130123_AUC: 0.50543818752728
130125_AUC: 0.5122395653804042
130129_AUC: 0.5097079125981342
130131_AUC: 0.5063788485452988
140307_AUC: 0.5091186741319937
140313_AUC: 0.5111818597138876
140316_AUC: 0.5189006261926927
140317_AUC: 0.5269429767941406
140321_AUC: 0.5157846813429187
140501_AUC: 0.5067210042557714
140505_AUC: 0.5059236309990233
140641_AUC: 0.5147655776912725
140691_AUC: 0.5095297519169797



In [69]:
sample_submission["130123"] = lgb_model_130123.y_pred
sample_submission["130125"] = lgb_model_130125.y_pred
sample_submission["130129"] = lgb_model_130129.y_pred
sample_submission["130131"] = lgb_model_130131.y_pred
sample_submission["140307"] = lgb_model_140307.y_pred
sample_submission["140313"] = lgb_model_140313.y_pred
sample_submission["140316"] = lgb_model_140316.y_pred
sample_submission["140317"] = lgb_model_140317.y_pred
sample_submission["140321"] = lgb_model_140321.y_pred
sample_submission["140501"] = lgb_model_140501.y_pred
sample_submission["140505"] = lgb_model_140505.y_pred
sample_submission["140641"] = lgb_model_140641.y_pred
sample_submission["140691"] = lgb_model_140691.y_pred

In [70]:
sample_submission.head()

,130123,130125,130129,130131,140307,140313,140316,140317,140321,140501,140505,140641,140691
0,0.125916,0.018495,0.065802,0.151879,0.087172,0.044379,0.016416,0.019307,0.012002,0.047892,0.076854,0.027323,0.025388
1,0.123771,0.017051,0.064819,0.149858,0.083424,0.044674,0.016716,0.014424,0.009615,0.046444,0.073613,0.025516,0.024517
2,0.131856,0.018427,0.065081,0.157233,0.086959,0.045294,0.015598,0.011010,0.010406,0.045974,0.075453,0.026174,0.024202
3,0.128921,0.019875,0.064486,0.153029,0.086791,0.046551,0.015550,0.013263,0.011894,0.043774,0.074882,0.025893,0.025456
4,0.130108,0.018902,0.064229,0.153422,0.085195,0.046187,0.015740,0.016352,0.009917,0.045064,0.075573,0.025852,0.024684


In [71]:
sample_submission.to_csv(f"{SAME_PATH}" + "/Users/td017/kaggle-pipeline/submission/sub_" + f"{CASE}"+"_second.csv",index=False)

In [72]:
# stack_df = pd.DataFrame({
#   "130123" :  lgb_model_130123.oof_pred,
#   "130125" :  lgb_model_130125.oof_pred,
#   "130129" :  lgb_model_130129.oof_pred,
#   "130131" :  lgb_model_130131.oof_pred,
#   "140307" :  lgb_model_140307.oof_pred,
#   "140313" :  lgb_model_140313.oof_pred,
#   "140316" :  lgb_model_140316.oof_pred,
#   "140317" :  lgb_model_140317.oof_pred,
#   "140321" :  lgb_model_140321.oof_pred,
#   "140501" :  lgb_model_140501.oof_pred,
#   "140505" :  lgb_model_140505.oof_pred,
#   "140641" :  lgb_model_140641.oof_pred,
#   "140691" :  lgb_model_140691.oof_pred,
# })

In [73]:
# case1_oof_pred = pd.read_csv(f"{SAME_PATH}" + "Users/td017/kaggle-pipeline/submission/sub_case1.csv")

In [74]:
# case1